Merge the motor and non-motor measures while also selecting only enrolled patients. Reconcile 'INFODT' errors using the signature form csv.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path = ''
non_motor = pd.read_csv(path+'non-motor.csv') #note that this is the processed data created by 'Non_Motor_Data_Processing.ipynb'
motor = pd.read_csv(path+'motor.csv') #note that this is the processed data created by 'Motor_Data_Processing.ipynb'
demographics = pd.read_csv(path+'demographics.csv') #note that this is the processed data created by 'Demographics_Data_Processing.ipynb'

In [ ]:
print(len(demographics))
demographics.head()

In [ ]:
pd_ids = demographics[(~demographics.ENROLL_DATE.isnull()) & (demographics.COHORT==1.0)].PATNO #PD cohort
hc_ids = demographics[(~demographics.ENROLL_DATE.isnull()) & (demographics.COHORT==2.0)].PATNO #HC cohort

In [ ]:
print(len(pd_ids), len(hc_ids))

In [ ]:
df = motor.merge(non_motor, how='outer', on=['PATNO','EVENT_ID','INFODT'])
df = df[(df.PATNO.isin(pd_ids)) | (df.PATNO.isin(hc_ids))]
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))


In [ ]:
df['PAG_NAME'].value_counts()

In [ ]:
# create a data frame taking only the on measurements when paired testing was performed
pd_on = pd.DataFrame(columns = df.columns)
reconcile_pn = []
reconcile_ei = []
for pn in pd_ids:
    for ei in df[df.PATNO==pn].EVENT_ID.unique():
        rec = df[(df.PATNO==pn) & (df.EVENT_ID==ei)]
        if len(rec) > 1:
            # one event has two or more records
            if (rec.PAG_NAME != 'NUPDRS3A').all():
                # print(pn, ei)
                reconcile_pn.append(pn)
                reconcile_ei.append(ei)
            else:
                # NUPDRS3A is ON State
                on_rec = df[(df.PATNO==pn) & (df.EVENT_ID==ei) & (df.PAG_NAME=='NUPDRS3A')]
                pd_on = pd.concat([pd_on,on_rec],ignore_index=True)
        else:
                pd_on = pd.concat([pd_on,rec],ignore_index=True)


In [ ]:
pd_on.drop_duplicates(inplace=True)
print(len(pd_on))

In [ ]:

interest_cols = [
'NP1COG',
'NP1HALL',
'NP1DPRS',
'NP1ANXS',
'NP1APAT',
'NP1DDS',
'NP1SLPN',
'NP1SLPD',
'NP1PAIN',
'NP1URIN',
'NP1CNST',
'NP1LTHD', # U1 scores
'NP1FATG',
'NP2SPCH',
'NP2SALV',
'NP2SWAL',
'NP2EAT',
'NP2DRES',
'NP2HYGN',
'NP2HWRT',
'NP2HOBB',
'NP2TURN',
'NP2TRMR',
'NP2RISE',
'NP2WALK',
'NP2FREZ', # u2 scores
'MSEADLG', # Modifier S and E overall score
'PAG_NAME', # U3, to see whether on or off
'DYSKPRES', # U3 3.19, whether dyskinesias persents
'DYSKIRAT', # U3 3.20
'NHY', # U3 3.21
'PDSTATE', # U3 which function is participants currently in, ON/OFF
'PDTRTMNT',# U3 is participants on medication or DBS
'NP3SPCH',
'NP3FACXP',
'NP3RIGN',
'NP3RIGRU',
'NP3RIGLU',
'NP3RIGRL',
'NP3RIGLL',
'NP3FTAPR',
'NP3FTAPL',
'NP3HMOVR',
'NP3HMOVL',
'NP3PRSPR',
'NP3PRSPL',
'NP3TTAPR',
'NP3TTAPL',
'NP3LGAGR',
'NP3LGAGL',
'NP3RISNG',
'NP3GAIT',
'NP3FRZGT',
'NP3PSTBL',
'NP3POSTR',
'NP3BRADY',
'NP3PTRMR',
'NP3PTRML',
'NP3KTRMR',
'NP3KTRML',
'NP3RTARU',
'NP3RTALU',
'NP3RTARL',
'NP3RTALL',
'NP3RTALJ',
'NP3RTCON',
'NP3TOT',
'SDMTOTAL', # Symbol Digit Modalities,  Neuropsychological Test, tot
'STAI_TOT', # State-Trait Anxiety Inventory for Adults tot 
'SFT_TOT', # Semantic Fluency tot
'SCOPA_AUT_TOT', #SCOPA-AUT, autonomic test, Lower score means better.
'REMSLEEP_TOT', # REM Sleep Behavior Questionnaire
'QUIP_A', # This is a questionnaire about gambling, buying, etc. It is a nerobehavioral questionnaire.
'QUIP_B',
'QUIP_C',
'QUIP_D',
'QUIP_E',
'UPSIT_TOT', # University_of_Pennsylvania_Smell_Identification_Test__UPSIT
'MoCA_score',
'LNS_TOT', # Letter Number Sequencing, total score
'HVLT_TOT_Recall', # Hopkins Verbal Learning Test， HVLT Immediate/Total Recall	Sum of HVLTRT1 - HVLTRT3
'HVLT_DCR_REC',
'HVLT_RETENTION',
'GDS_TOT', # The Geriatric Depression Scale (GDS) is a 30-item self-report assessment used to identify depression 
'GDS_Depressed', # if GDS tot >5
'ESS_TOT', # Epworth Sleepiness Scale
'ESS_Sleepy', # if ESS_ToT>10
'BJLOT_TOT', #  Benton Judgement of Line Orientation total score
'DATSCAN_CAUDATE_R', # 'CAUDATE_R',
'DATSCAN_CAUDATE_L', # 'CAUDATE_L',
'DATSCAN_PUTAMEN_R',# 'PUTAMEN_R',
'DATSCAN_PUTAMEN_L'# 'PUTAMEN_L']
]


In [ ]:
print(len(interest_cols))

In [ ]:
from tqdm import tqdm
def process_multi_rows(pat, visit, df):
    # print("Working on",pat,visit)
    
    if len(df)<=1: 
        print('yes')
#         print("Length of selection for ",pat,visit," is ",len(df))
        return df
    res_row=df.iloc[0]
#     info_dt = df_sig[(df_sig['PATNO']==pat) & (df_sig['EVENT_ID']==visit)]['INFODT'].values[0]
#     res_row['INFODT'] = pd.to_datetime(info_dt, format='%m/%Y')
#     res_row['INFODT'] = res_row['INFODT'].strftime('%m/%Y')

    for col in interest_cols:
            cur_col=df[col]
            for i in cur_col:
                if isinstance(i,str) or not np.isnan(i):
                    res_row[col]=i
                    break
    df = df.drop(df[(df['PATNO']==pat) & (df['EVENT_ID']==visit)].index)
    # df = df.append(res_row)
    df = pd.concat([df,res_row], ignore_index=True)

    return df

In [ ]:
for i in tqdm(range(len(reconcile_pn))):
    rec = df[(df.PATNO==reconcile_pn[i]) & (df.EVENT_ID==reconcile_ei[i])]
    test = process_multi_rows(reconcile_pn[i], reconcile_ei[i], rec)
    pd_on = pd.concat([pd_on,test], ignore_index=True)

In [ ]:
#create a dataframe of only healthy controls
hc = pd.DataFrame(columns = df.columns)
reconcile_pn = []
reconcile_ei = []
for pn in hc_ids:
    for ei in df[df.PATNO==pn].EVENT_ID.unique():
        rec = df[(df.PATNO==pn) & (df.EVENT_ID==ei)]
        if len(rec) > 1:
            # print(pn, ei)
            reconcile_pn.append(pn)
            reconcile_ei.append(ei)
        else:
            hc = pd.concat([hc, rec],ignore_index=True)

In [ ]:
for i in range(len(reconcile_pn)):
    rec = df[(df.PATNO==reconcile_pn[i]) & (df.EVENT_ID==reconcile_ei[i])]
    test = process_multi_rows(reconcile_pn[i], reconcile_ei[i], rec)
    hc = pd.concat([hc,test], ignore_index=True)

In [ ]:
print(len(hc))
print(len(hc['PATNO'].unique()))
pd_on = pd_on.drop(index=pd_on[pd_on.INFODT.isnull()].index)
print(len(pd_on))
print(len(pd_on['PATNO'].unique()))


In [ ]:
pd_on.to_csv(path+'pd_on.csv', index=False)
hc.to_csv(path+'hc.csv', index=False)